In [1]:
import torch
import json

In [2]:
import numpy as np

In [3]:
import torch.optim as optim

In [4]:
X_train = []
Y_train = []
train = []

X_test = []
Y_test = []
test = []

f = open('train_datapoints.json')
data_x = json.load(f)

designs = list(data_x.keys())
for i in range(len(data_x)):
    for j in data_x[designs[i]]:
        train.append([i, int(j), data_x[designs[i]][j][0], data_x[designs[i]][j][1], data_x[designs[i]][j][2], 
                        data_x[designs[i]][j][3], data_x[designs[i]][j][4]])
        
f = open('test_datapoints.json')
data_t = json.load(f)

for i in range(len(data_x)):
    for j in data_t[designs[i]]:
        test.append([i, int(j), data_t[designs[i]][j][0], data_t[designs[i]][j][1], data_t[designs[i]][j][2], 
                        data_t[designs[i]][j][3], data_t[designs[i]][j][4]])

In [5]:
train = np.array(train)

train = train/(train.max(axis=0)-train.min(axis=0))

In [6]:
test = np.array(test)

test = test/(test.max(axis=0)-train.min(axis=0))

In [ ]:
train

In [ ]:
train = np.array(train, dtype=np.float32)
test = np.array(test, dtype=np.float32)
print(train[0])

In [7]:
np.random.shuffle(train)

In [8]:
np.random.shuffle(test)

In [ ]:
len(train)

In [9]:
X_train = np.array(train[:, :-1])
Y_train = np.array(train[:, -1])

In [10]:
X_test = np.array(test[:, :-1])
Y_test = np.array(test[:, -1])

In [11]:
from torch.autograd import Variable
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super().__init__()
        self.l1 = torch.nn.Linear(inputSize, 16)
        self.ReLu1 = torch.nn.ReLU()
        self.l2 = torch.nn.Linear(16, 16)
        self.ReLu2 = torch.nn.ReLU()
        self.l3 = torch.nn.Linear(16, 16)
        self.ReLu3 = torch.nn.ReLU()
        self.l4 = torch.nn.Linear(16, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, x):
        out = self.l1(x)
        out = self.ReLu1(out)
        out = self.l2(out)
        out = self.ReLu2(out)
        out = self.l3(out)
        out = self.ReLu3(out)
        out = self.l4(out)
        out = self.sig(out)
        return out

In [12]:
X_train = np.array(X_train, dtype=np.float32)
Y_train = np.array(Y_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
Y_test = np.array(Y_test, dtype=np.float32)

In [16]:
inputDim = 6        # takes variable 'x' 
outputDim = 1       # takes variable 'y'
learningRate = 0.01 
epochs = 100

model = linearRegression(inputDim, outputDim)
##### For GPU #######
if torch.cuda.is_available():
    model.cuda()

In [17]:
criterion = torch.nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.7, 0.999), eps=1e-08, weight_decay=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [ ]:
for epoch in range(epochs):
    # Converting inputs and labels to Variable
    if torch.cuda.is_available():
        inputs = Variable(torch.from_numpy(X_train).cuda())
        labels = Variable(torch.from_numpy(Y_train).cuda())
    else:
        inputs = Variable(torch.from_numpy(X_train))
        labels = Variable(torch.from_numpy(Y_train))

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    outputs = model(inputs)

    # get loss for the predicted output
    loss = criterion(outputs, labels)
    print(loss)
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    print('epoch {}, loss {}'.format(epoch, loss.item()))

In [ ]:
import matplotlib.pyplot as plt

with torch.no_grad(): # we don't need gradients in the testing phase
    if torch.cuda.is_available():
        predicted = model(Variable(torch.from_numpy(X_train).cuda())).cpu().data.numpy()
    else:
        predicted = model(Variable(torch.from_numpy(X_train))).data.numpy()
    print(predicted)

plt.clf()
plt.plot(X_train[:,0], Y_train, 'go', label='True data', alpha=0.5)
plt.plot(X_train[:,0], predicted, 'o', label='Predictions', alpha=0.5)
plt.legend(loc='best')
plt.show()

In [ ]:
Y_train

In [ ]:
max(Y_train)

In [ ]:
arr = np.array([[1,2,3],[4,5,6],[7,8,9]])
from sklearn.preprocessing import normalize
p = normalize(arr, axis=1, norm='l1')

In [ ]:
Y_train

In [ ]:
arr

In [ ]:
q = normalize(arr, axis=0, norm='l1')

In [ ]:
q

In [ ]:
a = np.array([1, 4, 7])
a = a.reshape(-1,1)
normalize(a, axis=0, norm='l1')

In [18]:
for epoch in range(100):  # loop over the dataset multiple times
    trainloader = [X_train, Y_train]
    running_loss = 0.0
    for i in range(0, len(X_train), 128):
        # get the inputs; data is a list of [inputs, labels]
        if torch.cuda.is_available():
            inputs = Variable(torch.from_numpy(X_train[i:i+128]).cuda())
            labels = Variable(torch.from_numpy(Y_train[i:i+128]).cuda())
        else:
            inputs = Variable(torch.from_numpy(X_train[i:i+128]))
            labels = Variable(torch.from_numpy(Y_train[i:i+128]))

        inputs, labels = torch.tensor(inputs).unsqueeze(1).float(), torch.tensor(labels)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
#         print(inputs.shape)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        batch_size = len(labels)
    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / batch_size:.3f}')
    running_loss = 0.0
    scheduler.step()

# test_acc = check_accuracy(cnnWithBatchNormNet, Data(batch_size=batch_size, dataset_directory=dataset_directory).load_test_data(), 'forward')
# print('Finished Training with test acc: ', test_acc)

/tmp/ipykernel_575/3196261355.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs, labels = torch.tensor(inputs).unsqueeze(1).float(), torch.tensor(labels)


[1, 50305] loss: 4.183
[2, 50305] loss: 0.991
[3, 50305] loss: 0.981
[4, 50305] loss: 0.983
[5, 50305] loss: 0.989
[6, 50305] loss: 0.997
[7, 50305] loss: 1.006
[8, 50305] loss: 1.013
[9, 50305] loss: 1.020
[10, 50305] loss: 1.025
[11, 50305] loss: 1.030
[12, 50305] loss: 1.032
[13, 50305] loss: 1.035
[14, 50305] loss: 1.037
[15, 50305] loss: 1.040
[16, 50305] loss: 1.043
[17, 50305] loss: 1.045
[18, 50305] loss: 1.047
[19, 50305] loss: 1.049
[20, 50305] loss: 1.050
[21, 50305] loss: 1.052
[22, 50305] loss: 1.052
[23, 50305] loss: 1.052
[24, 50305] loss: 1.053
[25, 50305] loss: 1.053
[26, 50305] loss: 1.053
[27, 50305] loss: 1.054
[28, 50305] loss: 1.054
[29, 50305] loss: 1.054
[30, 50305] loss: 1.054
[31, 50305] loss: 1.055
[32, 50305] loss: 1.054
[33, 50305] loss: 1.054
[34, 50305] loss: 1.054
[35, 50305] loss: 1.054
[36, 50305] loss: 1.055
[37, 50305] loss: 1.055


KeyboardInterrupt: 

In [ ]:
trainloader = [X_train, Y_train]
for i in enumerate(trainloader, 32):
    print("00000", i[0], "111111", len(i[1]))

In [19]:
for i in range(1,10,3):
    print(i)

1
4
7
